In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [19]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = r'F:\DATA ENGINEERING\Course-3\ETL_GDP_Data\Countries_by_GDP.csv'

In [5]:
page = requests.get(url).text
data = BeautifulSoup(page ,'html.parser')
df = pd.DataFrame(columns=[table_attribs])
tables = data.find_all('tbody')
rows = tables[2].find_all('tr')
for row in rows:
     col = row.find_all('td')
     if len(col)!=0:
        if col[0].find('a') is not None and '—' not in col[2]:
            data_dict = {"Country":col[0].a.contents[0], "GDP_USD_millions": col[2].contents[0]}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
df

,"(Country,)","(GDP_USD_millions,)",Country,GDP_USD_millions
0,NaN,NaN,United States,"26,854,599"
1,NaN,NaN,China,"19,373,586"
2,NaN,NaN,Japan,"4,409,738"
3,NaN,NaN,Germany,"4,308,854"
4,NaN,NaN,India,"3,736,882"
...,...,...,...,...
186,NaN,NaN,Marshall Islands,291
187,NaN,NaN,Palau,262
188,NaN,NaN,Kiribati,248
189,NaN,NaN,Nauru,151


In [20]:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page ,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country":col[0].a.contents[0], "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df
d=extract(url, table_attribs)
d



,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151


In [21]:
def transform(df):
    GDP_list = df['GDP_USD_millions'].tolist()
    GDP_list = ([float("".join(x.split(','))) for x in GDP_list])
    GDP_list = [np.round(x/1000,2) for x in GDP_list]
    df["GDP_USD_millions"] = GDP_list
    df=df.rename(columns = {"GDP_USD_millions":"GDP_USD_billions"})
    return df

transform(d)


,Country,GDP_USD_billions
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88
...,...,...
186,Marshall Islands,0.29
187,Palau,0.26
188,Kiribati,0.25
189,Nauru,0.15


In [24]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)
load_to_csv(df, csv_path)


In [25]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
sql_connection = sqlite3.connect('World_Economies.db')
load_to_db(df, sql_connection, table_name)

In [29]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)
query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100


DatabaseError: Execution failed on sql 'SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100': no such column: GDP_USD_billions